In [86]:
# 用于测试集的划分
from sklearn.model_selection import train_test_split

# 进度条工具
from tqdm import tqdm

In [80]:
train_feature = train_merged_2.drop('label', axis=1)
train_label = train_merged_2['label']

# 为了防止过拟合，需要从训练集中划分出一个验证集来
X_train, X_test, y_train, y_test = train_test_split(train_feature, train_label, test_size=0.1)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

# 此数据集的测试集中没有label，仅用于提交结果，所以需要在训练完毕后对此集做推理运算，得出label
test_feature = test_merged_2

(3, 32)
(1, 32)
(3,)
(1,)


## 通过得到的特征向量与标签，用tensorflow搭建神经网络进行训练

In [43]:
import tensorflow as tf

In [ ]:
# 设计神经网络结构
def build_arch(inputs):
    layer1 = tf.layers.dense(inputs, 128, activation=tf.nn.relu)
    layer2 = tf.layers.dense(layer1, 128, activation=tf.nn.relu)
    layer3 = tf.layers.dense(layer2, 128, activation=tf.nn.relu)
    outputs = tf.layers.dense(layer3, 1, activation=tf.nn.relu)
    
    return outputs

In [ ]:
# 设计损失函数
def build_loss(outputs, labels):
    loss = tf.nn.softmax_cross_entropy_with_logits(outputs, labels)
    return loss

In [ ]:
# approximated AUC loss function
def build_AUC_loss(outputs, labels):
    
    posi_idx = tf.where(tf.equal(labels,1.0))
    neg_idx = tf.where(tf.equal(labels,-1.0))
    
    prdictions = tf.nn.softmax(outputs)
    # 注意以下数据的纬度
    posi_predict = tf.gather(prdictions, posi_idx)
    posi_size = tf.shape(posi_predict)[0]
    
    neg_predict = tf.gather(prdictions, neg_idx)
    neg_size = tf.shape(posi_predict)[0]
    
    gamma = 0.1
    power_p = 2
    posi_neg_diff = tf.reshape(
                        -(tf.matmul(posi_predict,tf.ones([1,neg_size])) -
                        tf.matmul(tf.ones([posi_size,1]), tf.reshape(neg_predict,[-1,neg_size]) ) - gamma),
                        [-1,1]
                               )
    posi_neg_diff = tf.where(tf.greater(posi_neg_diff,0),posi_neg_diff,tf.zeros([posi_size*neg_size,1]))
    posi_neg_diff = tf.pow(posi_neg_diff,power_p)
    
    loss_approx_auc = tf.reduce_mean(posi_neg_diff)
    return loss_approx_auc

In [ ]:
# 设计分数统计函数
def build_score(outputs, labels):
    score = 0
    return score

In [ ]:
# 获取上一次保存的模型
def get_last_state(logdir, num_batch):
    ckpt = tf.train.get_checkpoint_state(logdir)
    if ckpt and ckpt.model_checkpoint_path:
        # Restores from checkpoint
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        global_step = int(ckpt_name.split('-')[-1])
        last_epoch = global_step // num_batch
        last_step = global_step % num_batch
    else:
        global_step = 0
        last_epoch = 0
        last_step = 0
    return ckpt, global_step, last_epoch, last_step

### 构建计算图

In [ ]:
# 调试超参
lr = 0.1
batch_size = 128
epoch = 1
logdir = 'log/'
save_checkpoint_steps = 20
save_summaries_steps = 10
#AUC parameter
gammar = 0.1
power_p = 2

In [ ]:
tf.reset_default_graph()
outputs = build_arch(X_train)
loss = build_loss(outputs, y_train)
score = build_score(outputs, y_train)
opt = tf.train.AdamOptimizer(lr).minimize(loss)

### 训练

In [ ]:
num_batch = X_train.shape[0] // batch_size
ckpt, global_step, last_epoch, last_step = get_last_state(logdir, num_batch)
init_op = tf.global_variables_initializer()
saver = tf.train.Saver()
with tf.Session() as sess:
    sess.run(init_op)
    train_writer = tf.summary.FileWriter(logdir + '/train', sess.graph)
    
    if ckpt and ckpt.model_checkpoint_path:
        # 加载上次保存的模型
        saver.restore(sess, ckpt.model_checkpoint_path)
    # 计算图结构分析
    param_stats = tf.contrib.tfprof.model_analyzer.print_model_analysis(
            tf.get_default_graph(),
            tfprof_options=tf.contrib.tfprof.model_analyzer.TRAINABLE_VARS_PARAMS_STAT_OPTIONS)
    print('total_params: %d\n' % param_stats.total_parameters)
    
    for e in range(last_epoch, epoch):
        print('Training for epoch ' + str(epoch+1) + '/' + str(epoch) + ':')
        
        bar = tqdm(range(last_step, num_batch), initial=last_step, total=num_batch, ncols=100, leave=False,
                       unit='b')
        for _ in bar:
            if global_step % save_summaries_steps == 0:
                # train
                _, train_score, summary_str = sess.run(
                        [opt, score, summary])
                train_writer.add_summary(summary_str, global_step)
                bar.set_description('tr_acc:{}'.format(train_score))
            else:
                sess.run(opt)

            global_step += 1
            if global_step % save_checkpoint_steps == 0:
                saver.save(sess, logdir + '/model.ckpt', global_step=global_step)

    train_writer.close()